In [1]:
"""
Step1:
1.Read the Zillow ZHVI CSV.
2.Detect all monthly date columns (YYYY-MM-DD).
3.Convert the wide table into a long table (zip, month, zhvi).
4.Filter to keep only records from January 2023 and later.
5.Convert the month column from full date (e.g., 2023-01-31) to year-month format (e.g., 2023-01).
"""

'\nStep1:\n1.Read the Zillow ZHVI CSV.\n2.Detect all monthly date columns (YYYY-MM-DD).\n3.Convert the wide table into a long table (zip, month, zhvi).\n4.Filter to keep only records from January 2023 and later.\n5.Convert the month column from full date (e.g., 2023-01-31) to year-month format (e.g., 2023-01).\n'

In [8]:
import sys
from pathlib import Path

# Project root so "from src.acquire" and data paths work from any cwd
ROOT = Path.cwd()
while ROOT != ROOT.parent and not (ROOT / "pyproject.toml").exists():
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from src.acquire._loaders import load_all_newest
results = load_all_newest()

In [9]:
# Use newest raw files from load_all_newest() (run cell above first)
if "zillow" not in results or results["zillow"][1] is None:
    raise FileNotFoundError("No Zillow ZHVI CSV file found in 'data/raw/zillow/'. Please download or generate data.")
latest_zillow_path, zillow = results["zillow"]
print(f"Loading Zillow file: {latest_zillow_path.name} -> shape {zillow.shape}")

import re
import pandas as pd

# Identify columns in YYYY-MM-DD format
date_pattern = re.compile(r"^\d{4}-\d{2}-\d{2}$")
date_cols = [col for col in zillow.columns if date_pattern.match(str(col))]

if len(date_cols) == 0:
    raise ValueError("No monthly date columns found. Please check your Zillow column names.")

date_cols = sorted(date_cols)

# Build (zip, month, zhvi) long table
zillow["zip"] = zillow["RegionName"].astype(str).str.zfill(5)
zillow_small = zillow[["zip"] + date_cols].copy()

zillow_long = pd.melt(
    zillow_small,
    id_vars=["zip"],
    value_vars=date_cols,
    var_name="month",
    value_name="zhvi"
)
zillow_long["month"] = pd.to_datetime(zillow_long["month"], errors="coerce")

# Filter to keep >= 2023-01-01
zillow_long = zillow_long[zillow_long["month"] >= pd.Timestamp("2023-01-01")].copy()

# Convert month to Year-Month format (YYYY-MM)
zillow_long["month"] = zillow_long["month"].dt.to_period("M").astype(str)
zillow_long = zillow_long.sort_values(["zip", "month"]).reset_index(drop=True)

print(zillow_long.head(10))
print("Shape:", zillow_long.shape)

Loading Zillow file: zillow_zhvi_20260212_165121.csv -> shape (26307, 321)


C:\Users\jason\AppData\Local\Temp\ipykernel_8248\4293575548.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  zillow["zip"] = zillow["RegionName"].astype(str).str.zfill(5)


     zip    month           zhvi
0  01001  2023-01  284265.293218
1  01001  2023-02  285125.304634
2  01001  2023-03  286436.003321
3  01001  2023-04  288442.991612
4  01001  2023-05  290659.546155
5  01001  2023-06  292723.607850
6  01001  2023-07  294840.366054
7  01001  2023-08  297214.654496
8  01001  2023-09  299622.269960
9  01001  2023-10  301303.118047
Shape: (947052, 3)


In [10]:
"""
Step2:
1.Read the ACS CSV.
2.Keep the key socioeconomic columns, rename them to readable feature names, and create a few basic derived features (rates).
3.Standardize the ZIP key to a 5-digit string.
4.Merge ACS features into the Zillow long table on zip, and save the merged dataset for the next steps.
"""

'\nStep2:\n1.Read the ACS CSV.\n2.Keep the key socioeconomic columns, rename them to readable feature names, and create a few basic derived features (rates).\n3.Standardize the ZIP key to a 5-digit string.\n4.Merge ACS features into the Zillow long table on zip, and save the merged dataset for the next steps.\n'

In [11]:
# Standardize ZIP
zillow_long["zip"] = zillow_long["zip"].astype(str).str.zfill(5)

# Convert to datetime first
zillow_long["month"] = pd.to_datetime(zillow_long["month"], errors="coerce")

# ====== CHANGE HERE ======
# Convert from full date (e.g., 2023-01-31) to Year-Month format (e.g., 2023-01)
zillow_long["month"] = zillow_long["month"].dt.to_period("M").astype(str)
# =========================

# ====== 1) Read ACS raw data (newest from data/raw/acs/) ======
if "acs" not in results or results["acs"][1] is None:
    raise FileNotFoundError("No ACS file found in data/raw/acs/. Run the ACS acquirer first.")
_, acs = results["acs"]
acs = acs.copy()

# ====== 2) Select key ACS columns ======
keep_cols = [
    "zip code tabulation area",
    "B01003_001E",
    "B19013_001E",
    "B17001_001E",
    "B17001_002E",
    "B23025_003E",
    "B23025_005E",
    "B15003_022E",
    "B15003_023E",
    "B15003_024E",
    "B15003_025E",
]

acs_small = acs[keep_cols].copy()

# Rename columns
acs_small = acs_small.rename(columns={
    "zip code tabulation area": "zip",
    "B01003_001E": "population",
    "B19013_001E": "median_income",
    "B17001_001E": "poverty_base",
    "B17001_002E": "poverty_count",
    "B23025_003E": "labor_force",
    "B23025_005E": "unemployed",
    "B15003_022E": "edu_bachelors",
    "B15003_023E": "edu_masters",
    "B15003_024E": "edu_professional",
    "B15003_025E": "edu_doctorate",
})

# Standardize ZIP
acs_small["zip"] = acs_small["zip"].astype(str).str.zfill(5)

# Derived features
acs_small["poverty_rate"] = acs_small["poverty_count"] / acs_small["poverty_base"]
acs_small["unemployment_rate"] = acs_small["unemployed"] / acs_small["labor_force"]
acs_small["higher_ed_count"] = (
    acs_small["edu_bachelors"]
    + acs_small["edu_masters"]
    + acs_small["edu_professional"]
    + acs_small["edu_doctorate"]
)

# Merge
zillow_acs = zillow_long.merge(acs_small, on="zip", how="left")

print(zillow_acs.head())
print("Shape:", zillow_acs.shape)

     zip    month           zhvi  population  median_income  poverty_base  \
0  01001  2023-01  284265.293218     16136.0        71924.0       15624.0   
1  01001  2023-02  285125.304634     16136.0        71924.0       15624.0   
2  01001  2023-03  286436.003321     16136.0        71924.0       15624.0   
3  01001  2023-04  288442.991612     16136.0        71924.0       15624.0   
4  01001  2023-05  290659.546155     16136.0        71924.0       15624.0   

   poverty_count  labor_force  unemployed  edu_bachelors  edu_masters  \
0         1130.0       8511.0       441.0         2660.0       1256.0   
1         1130.0       8511.0       441.0         2660.0       1256.0   
2         1130.0       8511.0       441.0         2660.0       1256.0   
3         1130.0       8511.0       441.0         2660.0       1256.0   
4         1130.0       8511.0       441.0         2660.0       1256.0   

   edu_professional  edu_doctorate  poverty_rate  unemployment_rate  \
0             223.0        

In [12]:
"""
Step3:
1.Spatial Assignment of ZIP Codes:
The crime dataset contains incident-level records with latitude and longitude but no ZIP code. The MODZCTA dataset provides polygon boundaries for ZIP areas. We converted the the_geom field into polygon geometries and transformed each crime record into a geographic point. A point-in-polygon spatial join was performed to assign a ZIP code to each crime event.
2. Temporal Standardization:
The cmplnt_fr_dt field was converted to datetime format. Dates were transformed into a Year-Month format (YYYY-MM) to match the monthly structure of the housing dataset.
3. Monthly ZIP-Level Aggregation:
Crime records were grouped by (zip, month). The total number of incidents per ZIP per month was computed as crime_count.
4. Integration with Main Dataset:
The ZIP-month crime counts were merged into zillow_acs using a left join on (zip, month). Missing values were replaced with 0 to indicate no recorded crime for that ZIP-month.
"""

'\nStep3:\n1.Spatial Assignment of ZIP Codes:\nThe crime dataset contains incident-level records with latitude and longitude but no ZIP code. The MODZCTA dataset provides polygon boundaries for ZIP areas. We converted the the_geom field into polygon geometries and transformed each crime record into a geographic point. A point-in-polygon spatial join was performed to assign a ZIP code to each crime event.\n2. Temporal Standardization:\nThe cmplnt_fr_dt field was converted to datetime format. Dates were transformed into a Year-Month format (YYYY-MM) to match the monthly structure of the housing dataset.\n3. Monthly ZIP-Level Aggregation:\nCrime records were grouped by (zip, month). The total number of incidents per ZIP per month was computed as crime_count.\n4. Integration with Main Dataset:\nThe ZIP-month crime counts were merged into zillow_acs using a left join on (zip, month). Missing values were replaced with 0 to indicate no recorded crime for that ZIP-month.\n'

In [16]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

# Inputs: newest crime from data/raw/nyc_crime/, MODZCTA from data/raw/geo/
if "nyc_crime" not in results or results["nyc_crime"][1] is None:
    raise FileNotFoundError("No NYC crime file found in data/raw/nyc_crime/. Run the nyc_crime acquirer first.")
_, crime = results["nyc_crime"]
crime = crime.copy()

geo_dir = ROOT / "data" / "raw" / "geo"
modzcta_files = list(geo_dir.glob("*MODZCTA*.csv")) if geo_dir.exists() else []
if not modzcta_files:
    modzcta_files = list(geo_dir.glob("*.csv")) if geo_dir.exists() else []
modzcta_path = max(modzcta_files, key=lambda p: p.stat().st_mtime) if modzcta_files else None
if modzcta_path is None:
    raise FileNotFoundError("No MODZCTA CSV found in data/raw/geo/. Place Modified_Zip_Code_Tabulation_Areas__MODZCTA_.csv there.")

# df is already in memory
df = zillow_acs.copy()

# Make sure keys are standardized
df["zip"] = df["zip"].astype(str).str.zfill(5)
df["month"] = df["month"].astype(str)  # should already be 'YYYY-MM'


# =========================================================
# STEP 1) Assign ZIP (MODZCTA) to each crime using lat/lon + polygons
# =========================================================

# 1.1 Read MODZCTA polygons
zcta = pd.read_csv(modzcta_path)

zcta_small = zcta[["MODZCTA", "the_geom"]].copy()
zcta_small["zip"] = zcta_small["MODZCTA"].astype(str).str.zfill(5)

# Parse WKT -> geometry
zcta_small = zcta_small.dropna(subset=["the_geom"])
zcta_small["geometry"] = zcta_small["the_geom"].apply(wkt.loads)

zcta_gdf = gpd.GeoDataFrame(
    zcta_small[["zip", "geometry"]],
    geometry="geometry",
    crs="EPSG:4326"  # lon/lat
)

# 1.2 Crime points (already loaded from results; only need lat/lon + date)
needed_cols = ["cmplnt_fr_dt", "latitude", "longitude"]
crime_small = crime[needed_cols].copy()

# Convert lat/lon to numeric
crime_small["latitude"] = pd.to_numeric(crime_small["latitude"], errors="coerce")
crime_small["longitude"] = pd.to_numeric(crime_small["longitude"], errors="coerce")
crime_small = crime_small.dropna(subset=["latitude", "longitude"])

# ---- Auto-fix possible swapped lat/lon ----
# If many "latitude" values look like -73 (should be lon), swap them.
lat_looks_like_lon = (crime_small["latitude"].between(-75, -72)).mean()
lon_looks_like_lat = (crime_small["longitude"].between(40, 41.5)).mean()

if lat_looks_like_lon > 0.5 and lon_looks_like_lat > 0.5:
    # swap columns
    tmp = crime_small["latitude"].copy()
    crime_small["latitude"] = crime_small["longitude"]
    crime_small["longitude"] = tmp
    print("Detected swapped lat/lon -> swapped back.")

# ---- Keep only points in NYC-ish bounding box (helps avoid bad coords) ----
crime_small = crime_small[
    crime_small["longitude"].between(-75, -72) &
    crime_small["latitude"].between(40.0, 41.2)
].copy()

# Convert date -> datetime (we will need it later)
crime_small["cmplnt_fr_dt"] = pd.to_datetime(crime_small["cmplnt_fr_dt"], errors="coerce")
crime_small = crime_small.dropna(subset=["cmplnt_fr_dt"])

# Build GeoDataFrame for points
crime_gdf = gpd.GeoDataFrame(
    crime_small,
    geometry=gpd.points_from_xy(crime_small["longitude"], crime_small["latitude"]),
    crs="EPSG:4326"
)

# 1.3 Spatial join: point-in-polygon
crime_joined = gpd.sjoin(crime_gdf, zcta_gdf, how="inner", predicate="intersects")

print("Crime points after cleaning:", len(crime_gdf))
print("ZCTA polygons:", len(zcta_gdf))
print("Matched crimes (joined):", len(crime_joined))

# Keep only what we need going forward: assigned zip + date
crime_with_zip = crime_joined[["zip", "cmplnt_fr_dt"]].copy()

Crime points after cleaning: 577674
ZCTA polygons: 178
Matched crimes (joined): 577639


In [17]:
# STEP 2) Count crimes per (zip, month)

# Create month key as 'YYYY-MM' (same format as your df)
crime_with_zip["month"] = crime_with_zip["cmplnt_fr_dt"].dt.to_period("M").astype(str)

crime_counts = (
    crime_with_zip.groupby(["zip", "month"])
    .size()
    .reset_index(name="crime_count")
)

print("Crime_counts rows:", len(crime_counts))
print(crime_counts.head())

Crime_counts rows: 4489
     zip    month  crime_count
0  10001  2023-02            1
1  10001  2023-03            1
2  10001  2023-10            3
3  10001  2023-11            2
4  10001  2024-01            3


In [18]:
# STEP 3) Merge crime_count into zillow_acs on (zip, month)
df3 = df.merge(crime_counts, on=["zip", "month"], how="left")

# Replace NaN in crime_count with 0
df3["crime_count"] = df3["crime_count"].fillna(0)

print("Non-NaN crime_count:", df3["crime_count"].notna().sum())
print("Total rows:", len(df3))
print(df3.head(10))

Non-NaN crime_count: 947052
Total rows: 947052
     zip    month           zhvi  population  median_income  poverty_base  \
0  01001  2023-01  284265.293218     16136.0        71924.0       15624.0   
1  01001  2023-02  285125.304634     16136.0        71924.0       15624.0   
2  01001  2023-03  286436.003321     16136.0        71924.0       15624.0   
3  01001  2023-04  288442.991612     16136.0        71924.0       15624.0   
4  01001  2023-05  290659.546155     16136.0        71924.0       15624.0   
5  01001  2023-06  292723.607850     16136.0        71924.0       15624.0   
6  01001  2023-07  294840.366054     16136.0        71924.0       15624.0   
7  01001  2023-08  297214.654496     16136.0        71924.0       15624.0   
8  01001  2023-09  299622.269960     16136.0        71924.0       15624.0   
9  01001  2023-10  301303.118047     16136.0        71924.0       15624.0   

   poverty_count  labor_force  unemployed  edu_bachelors  edu_masters  \
0         1130.0       8511.0   

In [19]:
"""
Step4:
1. Convert the date column to datetime format. If the FRED data were daily or weekly, values were aggregated to the monthly level (e.g., monthly average).
2. Merge the monthly macroeconomic variables into df3 (ZIP-month dataset) using a left join on month.

"""

'\nStep4:\n1. Convert the date column to datetime format. If the FRED data were daily or weekly, values were aggregated to the monthly level (e.g., monthly average).\n2. Merge the monthly macroeconomic variables into df3 (ZIP-month dataset) using a left join on month.\n\n'

In [20]:
df = df3.copy()

# Ensure month format is string 'YYYY-MM'
df["month"] = df["month"].astype(str)

# 1) Read FRED data (newest from data/raw/fred/)
if "fred" not in results or results["fred"][1] is None:
    raise FileNotFoundError("No FRED file found in data/raw/fred/. Run the fred acquirer first.")
_, fred = results["fred"]
fred = fred.copy()

# Check column names
print("FRED columns:", fred.columns)

fred["date"] = pd.to_datetime(fred["date"], errors="coerce")

# 2) Convert to Year-Month
fred["month"] = fred["date"].dt.to_period("M").astype(str)

# 3) Aggregate to monthly level (if needed)
fred_month = (
    fred.groupby("month")
    .mean(numeric_only=True)
    .reset_index()
)

# 4) Merge with main dataset
df4 = df.merge(fred_month, on="month", how="left")

print("Shape:", df4.shape)
print(df4.head())

FRED columns: Index(['date', 'series_id', 'value'], dtype='str')
Shape: (947052, 18)
     zip    month           zhvi  population  median_income  poverty_base  \
0  01001  2023-01  284265.293218     16136.0        71924.0       15624.0   
1  01001  2023-02  285125.304634     16136.0        71924.0       15624.0   
2  01001  2023-03  286436.003321     16136.0        71924.0       15624.0   
3  01001  2023-04  288442.991612     16136.0        71924.0       15624.0   
4  01001  2023-05  290659.546155     16136.0        71924.0       15624.0   

   poverty_count  labor_force  unemployed  edu_bachelors  edu_masters  \
0         1130.0       8511.0       441.0         2660.0       1256.0   
1         1130.0       8511.0       441.0         2660.0       1256.0   
2         1130.0       8511.0       441.0         2660.0       1256.0   
3         1130.0       8511.0       441.0         2660.0       1256.0   
4         1130.0       8511.0       441.0         2660.0       1256.0   

   edu_profes

In [21]:
"""
Step6:
1. Assess the missing value. All variables had a missing ratio below 1%, with the highest being approximately 0.5% for zhvi.
2. Remove the missing values. All rows containing missing values were dropped from the dataset.
"""

'\nStep6:\n1. Assess the missing value. All variables had a missing ratio below 1%, with the highest being approximately 0.5% for zhvi.\n2. Remove the missing values. All rows containing missing values were dropped from the dataset.\n'

In [22]:
# Missing data percent of each column
df = df4.copy()
missing_count = df.isna().sum()
missing_ratio = df.isna().mean()
missing_summary = pd.DataFrame({
    "missing_count": missing_count,
    "missing_ratio": missing_ratio
})

missing_summary = missing_summary.sort_values(by="missing_ratio", ascending=False)

print(missing_summary)

                   missing_count  missing_ratio
zhvi                        4930       0.005206
unemployment_rate           1224       0.001292
poverty_rate                 504       0.000532
unemployed                   324       0.000342
edu_bachelors                324       0.000342
population                   324       0.000342
poverty_count                324       0.000342
labor_force                  324       0.000342
median_income                324       0.000342
poverty_base                 324       0.000342
edu_doctorate                324       0.000342
higher_ed_count              324       0.000342
edu_professional             324       0.000342
edu_masters                  324       0.000342
month                          0       0.000000
zip                            0       0.000000
crime_count                    0       0.000000
value                          0       0.000000


In [25]:
# Save final preprocessed panel to data/processed/ (uses ROOT from cell 1)
out_dir = ROOT / "data" / "processed"
out_dir.mkdir(parents=True, exist_ok=True)
out_parquet = out_dir / "model_data.parquet"
out_csv = out_dir / "model_data.csv"

df_clean.to_parquet(out_parquet, index=False)
df_clean.to_csv(out_csv, index=False)

print(f"Saved preprocessed dataset:")
print(f"  Parquet: {out_parquet}")
print(f"  CSV:     {out_csv}")
print(f"  Shape:   {df_clean.shape}")

Saved preprocessed dataset:
  Parquet: c:\jason\Columbia_stuff\2025-2026\DS\project1\data\processed\model_data.parquet
  CSV:     c:\jason\Columbia_stuff\2025-2026\DS\project1\data\processed\model_data.csv
  Shape:   (940922, 18)


In [23]:
df_clean = df.dropna().copy()

print("Before drop:", len(df))
print("After drop:", len(df_clean))
print("Dropped rows:", len(df) - len(df_clean))

print("Remaining missing values:")
print(df_clean.isna().sum())

Before drop: 947052
After drop: 940922
Dropped rows: 6130
Remaining missing values:
zip                  0
month                0
zhvi                 0
population           0
median_income        0
poverty_base         0
poverty_count        0
labor_force          0
unemployed           0
edu_bachelors        0
edu_masters          0
edu_professional     0
edu_doctorate        0
poverty_rate         0
unemployment_rate    0
higher_ed_count      0
crime_count          0
value                0
dtype: int64


In [14]:
"""
Step7:
1.Descriptive Statistics Review:
Summary statistics and percentile distributions (1%, 5%, 95%, 99%) were examined for all numerical variables to identify extreme values.
2.Hard-Rule Validation:
Logical constraints were applied to detect impossible values (e.g., negative income, negative population, rates outside the [0,1] range).
3.IQR-Based Detection:
The Interquartile Range (IQR) method was used to identify extreme observations outside [Q1−1.5×IQR,Q3+1.5×IQR]
4.Z-Score Detection:
Standardized Z-scores were calculated, and observations with ∣z∣>3 were flagged as potential outliers.
"""

'\nStep7:\n1.Descriptive Statistics Review:\nSummary statistics and percentile distributions (1%, 5%, 95%, 99%) were examined for all numerical variables to identify extreme values.\n2.Hard-Rule Validation:\nLogical constraints were applied to detect impossible values (e.g., negative income, negative population, rates outside the [0,1] range).\n3.IQR-Based Detection:\nThe Interquartile Range (IQR) method was used to identify extreme observations outside [Q1−1.5×IQR,Q3+1.5×IQR]\n4.Z-Score Detection:\nStandardized Z-scores were calculated, and observations with ∣z∣>3 were flagged as potential outliers.\n'

In [24]:
# outlier detectives

import numpy as np

df = df_clean.copy()

# ====== 1) Pick numeric columns (exclude keys) ======
# We only run outlier checks on numeric columns.
key_cols = ["zip", "month"]
num_cols = []
for c in df.columns:
    if c in key_cols:
        continue
    if pd.api.types.is_numeric_dtype(df[c]):
        num_cols.append(c)

print("Numeric columns for outlier check:", num_cols)

# ====== 2) Basic descriptive stats (quick scan) ======
desc = df[num_cols].describe(percentiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]).T
print("\n=== Descriptive stats (with percentiles) ===")
print(desc)

# ====== 3) Hard-rule checks (impossible / suspicious values) ======
hard_rules = {
    "zhvi": lambda s: (s <= 0),
    "population": lambda s: (s < 0),
    "median_income": lambda s: (s < 0),
    "crime_count": lambda s: (s < 0),
    "poverty_rate": lambda s: (s < 0) | (s > 1),
    "unemployment_rate": lambda s: (s < 0) | (s > 1),
}

hard_summary = []
for col, rule_fn in hard_rules.items():
    if col in df.columns:
        mask = rule_fn(df[col])
        hard_summary.append({
            "column": col,
            "hard_outlier_count": int(mask.sum()),
            "hard_outlier_ratio": float(mask.mean())
        })

hard_summary = pd.DataFrame(hard_summary).sort_values("hard_outlier_count", ascending=False)
print("\n=== Hard-rule outliers ===")
print(hard_summary)

# ====== 4) IQR outlier detection (robust) ======
iqr_summary = []
iqr_masks = {}  # store masks to inspect rows later

for col in num_cols:
    s = df[col].dropna()
    if len(s) == 0:
        continue

    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    iqr = q3 - q1

    # If IQR is 0 (constant-ish column), skip to avoid division issues
    if pd.isna(iqr) or iqr == 0:
        iqr_summary.append({
            "column": col,
            "iqr_lower": np.nan,
            "iqr_upper": np.nan,
            "iqr_outlier_count": 0,
            "iqr_outlier_ratio": 0.0
        })
        iqr_masks[col] = pd.Series(False, index=df.index)
        continue

    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr

    mask = (df[col] < lower) | (df[col] > upper)
    iqr_masks[col] = mask

    iqr_summary.append({
        "column": col,
        "iqr_lower": float(lower),
        "iqr_upper": float(upper),
        "iqr_outlier_count": int(mask.sum()),
        "iqr_outlier_ratio": float(mask.mean())
    })

iqr_summary = pd.DataFrame(iqr_summary).sort_values("iqr_outlier_count", ascending=False)
print("\n=== IQR outliers (sorted) ===")
print(iqr_summary)

# ====== 5) Z-score outlier detection (|z| > 3) ======
z_summary = []
z_masks = {}

for col in num_cols:
    s = df[col]
    mean = s.mean()
    std = s.std()

    # If std is 0 or NaN, no z-score outliers
    if pd.isna(std) or std == 0:
        z_masks[col] = pd.Series(False, index=df.index)
        z_summary.append({
            "column": col,
            "z_outlier_count": 0,
            "z_outlier_ratio": 0.0
        })
        continue

    z = (s - mean) / std
    mask = z.abs() > 3
    z_masks[col] = mask

    z_summary.append({
        "column": col,
        "z_outlier_count": int(mask.sum()),
        "z_outlier_ratio": float(mask.mean())
    })

z_summary = pd.DataFrame(z_summary).sort_values("z_outlier_count", ascending=False)
print("\n=== Z-score outliers (|z|>3) ===")
print(z_summary)

# ====== 6) (Optional) Inspect top extreme rows for key columns ======
# This helps to quickly see which zip-month combos are extreme.
cols_to_inspect = ["zhvi", "crime_count", "median_income", "population"]
cols_to_inspect = [c for c in cols_to_inspect if c in df.columns]

print("\n=== Top extremes (highest and lowest) for selected columns ===")
for col in cols_to_inspect:
    print(f"\n--- Column: {col} (Top 10 high) ---")
    print(df.sort_values(col, ascending=False)[["zip","month",col]].head(10))
    print(f"--- Column: {col} (Top 10 low) ---")
    print(df.sort_values(col, ascending=True)[["zip","month",col]].head(10))

# ====== 7) (Optional) Create a combined outlier flag (for review only) ======
flag_cols = [c for c in ["zhvi", "crime_count"] if c in df.columns]
df_outlier_flag = df.copy()

df_outlier_flag["outlier_flag"] = False
for c in flag_cols:
    df_outlier_flag["outlier_flag"] = df_outlier_flag["outlier_flag"] | iqr_masks[c]

print("\nRows flagged as outliers (IQR on selected cols):", df_outlier_flag["outlier_flag"].sum())

Numeric columns for outlier check: ['zhvi', 'population', 'median_income', 'poverty_base', 'poverty_count', 'labor_force', 'unemployed', 'edu_bachelors', 'edu_masters', 'edu_professional', 'edu_doctorate', 'poverty_rate', 'unemployment_rate', 'higher_ed_count', 'crime_count', 'value']

=== Descriptive stats (with percentiles) ===
                      count          mean           std           min  \
zhvi               940922.0  3.432065e+05  3.027502e+05  2.589778e+04   
population         940922.0  1.254596e+04  1.593857e+04  4.000000e+00   
median_income      940922.0 -1.079581e+07  8.445204e+07 -6.666667e+08   
poverty_base       940922.0  1.228761e+04  1.568268e+04  4.000000e+00   
poverty_count      940922.0  1.522285e+03  2.430733e+03  0.000000e+00   
labor_force        940922.0  6.384988e+03  8.375076e+03  2.000000e+00   
unemployed         940922.0  3.305384e+02  5.148247e+02  0.000000e+00   
edu_bachelors      940922.0  1.846570e+03  2.789274e+03  0.000000e+00   
edu_masters